# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv(".env")

False

## Project client

In [ ]:
endpoint = os.getenv("PROJECT_ENDPOINT")
model = os.getenv("MODEL_DEPLOYMENT_NAME")
credential = DefaultAzureCredential()

agents_client = AgentsClient(endpoint=endpoint, credential=credential)

## Create agent

In [4]:
agent = project_client.create_agent(
    model=model,
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_YE9x6ypds6YmyCpBxP0T1bcw


In [5]:
prompt = "What is the best time to visit sydney?"

In [6]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_Xoi2CLPuftDWb2zYaOg286ik


In [7]:
# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content=prompt)
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_TJemQ76EhNr2kf1zmBNwghF9


In [8]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_Pnv7mDyqkb1FPFjTvRgTCmPb


In [9]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [10]:
run.usage

{'prompt_tokens': 33, 'completion_tokens': 338, 'total_tokens': 371, 'prompt_token_details': {'cached_tokens': 0}}

In [11]:
# [START list_messages]
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")
# [END list_messages]

MessageRole.USER: What is the best time to visit sydney?
MessageRole.AGENT: The **best time to visit Sydney** largely depends on your preferences for weather, crowds, and events, but in general, the ideal periods are:

### 1. **Spring (September to November)**
- **Weather:** Mild and pleasant (average temperatures 15-22°C/59-72°F).
- **Crowds:** Fewer tourists than in peak summer.
- **Nature:** Gardens and parks are in full bloom.
- **Events:** Sydney Fringe Festival, Sculpture by the Sea.

### 2. **Autumn (March to May)**
- **Weather:** Warm, comfortable, and drier (average temperatures 14-22°C/57-72°F).
- **Crowds:** Less busy than December-February.
- **Scenery:** Stunning autumn foliage in parks and harbours.
- **Events:** Sydney Royal Easter Show, Vivid Sydney (late May-June).

### 3. **Other Considerations**
- **Summer (December to February)**: Great for beach activities but can be hot (up to 26°C/79°F or higher), humid, and crowded due to holidays.
- **Winter (June to August)**:

## Clean Up: Delete Agent

In [12]:
project_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_YE9x6ypds6YmyCpBxP0T1bcw
